# Tweem.io Model Persistence

Calibrated model objects to be saved to postgres database table  

### Imports

In [1]:
import os
import pickle

In [2]:
import assembly
from assembly import config as asmbl_config
from assembly import db as asmbl_db
from assembly import models as asmbl_models

### Environment Settings

In [3]:
#  CD out to main app directory root, 
#  This is needed by app context to initialize properly
os.chdir('..')

In [4]:
#  Initialize environment
#  Set env to appropriate location
os.environ['ASSEMBLY_ENV'] = 'Development'
app = assembly.Assembly.init(__name__, {'default': []})

### Inputs

In [5]:
#  Directory where calibrated models are saved locally
MODEL_BASE_DIR = '/Users/liangjh/workspace/tweemio-api/data/models'

In [6]:
#  Calibration persistence parameters
#  trumpian, dempres2020, techbiz, entertainment, comedians
group = 'trumpian'

In [9]:
old_version = 1
new_version = 1

### Persist to Database

In [ ]:
#  Archive old / prior versions, to allow for rollback (if needed)
old_models = asmbl_models.TwmSnModel.query().filter(asmbl_models.TwmSnModel.active == True and asmbl_models.TwmSnModel.grp == group)
for model in old_models:
    model.update(active=False)       

In [ ]:
# for group in asmbl_config['SIMILARITY_COMPARISONS'].keys():
#     print(f'processing models for group: {group}')

In [ ]:
#  Save similarity model, for this group
with open(f'{MODEL_BASE_DIR}/{group}-mnbv1-similarity.pik', 'rb') as model_file:
    mdl_pkl = pickle.load(model_file)

asmbl_models.TwmSnModel.create(
    model_name='mnbv1-similarity', version=1, active=True, grp=group, pckl=mdl_pkl)

In [8]:
#  Save Vectorizer for this group
with open(f'{MODEL_BASE_DIR}/{group}-mnbv1-vectorizer.pik', 'rb') as model_file:
    mdl_pkl = pickle.load(model_file)

asmbl_models.TwmSnModel.create(
        model_name='mnbv1-vectorizer', version=1, active=True, grp=group, pckl=mdl_pkl)

processing models for group: trumpian
processing models for group: antitrump
processing models for group: prominentdems
processing models for group: techbiz
processing models for group: entertainment
processing models for group: comedians
